# Sessões da Câmara dos Deputados 🇧🇷

[Documentação oficial](http://www.camara.leg.br/internet/plenario/result/votacao/Layout_ArquivosTXT_presencas_vota%C3%A7%C3%A3o_exportados.pdf)

**Nomes dos arquivos**

```
HEaabbcddde000000.TXT presenças - cabeçalho
LPaabbcddde000000.TXT presenças - detalhes
HEaabbcdddeffffff.TXT votos - cabeçalho
LVaabbcdddeffffff.TXT votos - detalhes
```

## Cabeçalhos

| Linha presenças | Descrição presenças                              | Linha votação | Descrição votação |
|--------------|-----------------------------------------------|---------------|-------------------|
| **aabbcddde**    | conforme a nomenclatura do cabeçalho          | **aabbcddde**     | conforme nomenclatura do header  |
| **000000**       |                                               | **ffffff**        | seqüencial da votação            |
| **dd/mm/aaaa**   | data de término da sessão                     | **dd/mm/aaaa**    | data final da votação            |
| **hh:mm:ss**     | hora de término da sessão                     | **hh:mm:ss**      | hora final da votação            |
| **xxx**          | nome do 1º presidente da Sessão (40 posições) | **xxx**           | nome do 1º presidente da Votação |
| **000**          |                                               | **zzz**           | total de votos SIM               |
| **000**          |                                               | **vvv**           | total de votos NÃO               |
| **000**          |                                               | **www**           | total de votos ABSTENÇÃO         |
| **000**          |                                               | **ggg**           | total de votos OBSTRUÇÃO         |
| **000**          |                                               | **hhh**           | total de votos BRANCO            |
| **000**          |                                               | **iii**           | total de votos do presidente     |
| **000**          |                                               | **jjj**           | total de votantes                |
| **yyy**          | total de presentes                            | **kkk**           | nome da proposição               |

In [1]:
import glob
import numpy as np
import pandas as pd

paths = glob.glob('../data/sources/sessions/**/HE*')
headers = pd.DataFrame()
dtypes = {
    'header': np.str,
    'vote_number': np.str,
    'ending_date': np.str,
    'ending_time': np.str,
    'first_president': 'category',
    'yes_votes': np.int,
    'no_votes': np.int,
    'abstention_votes': np.int,
    'obstruction_votes': np.int,
    'blank_votes': np.int,
    'president_votes': np.int,
    'votes': np.int,
    'name': np.str,
}
for path in paths:
    with open(path, encoding='iso-8859-1') as file:
        attributes = file.read().split('\n')
        attributes = [attr.strip() for attr in attributes]
        row = pd.Series(dict(zip(dtypes.keys(), attributes[:13])))
        row['term'] = path.split('/')[-2]
        headers = headers.append(row, ignore_index=True)

Definir tipos corretos para cada uma das colunas.

In [2]:
for col, col_type in dtypes.items():
    headers[col] = headers[col].astype(col_type)

Criar uma coluna contendo data e hora do término da sessão (ou da votação, dependendo da linha).

In [3]:
headers['ending_time'] = pd.to_datetime(
    headers['ending_date'] + ' ' + headers['ending_time'], dayfirst=True)
headers.drop('ending_date', axis=1, inplace=True)

## Código do cabeçalho

**aabbcddde**

| Código | Descrição |
|--------|-----------|
| aa     | CD (sessão da Câmara) ou CC (sessão do Congresso – Câmara) ou SF (sessão do Congresso – Senado) |
| bb     | Número da Sessão legislativa (2 posições) |
| c      | O (sessão legislativa ordinária) ou E (sessão legislativa extraordinária) |
| ddd    | Número da Sessão (3 posições) |
| e      | O (sessão ordinária) ou E (sessão extraordinária) |
| ffffff | seqüencial da votação (6 posições) |

In [4]:
headers.head()

,abstention_votes,blank_votes,ending_time,first_president,header,name,no_votes,obstruction_votes,president_votes,term,vote_number,votes,yes_votes
0,0,0,2017-09-04 20:27:45,CARLOS MANATO,CD03O242E,345,0,0,0,2015,000000,0,0
1,0,0,2015-06-23 20:43:45,DANIEL VILELA,CD01O165E,319,0,0,0,2015,000000,0,0
2,3,0,2016-10-05 16:09:09,BETO MANSUR,CD02O241E,PL Nº 4567/2016 - REQUERIMENTO DE ADIAMENTO ...,262,62,1,2015,007188,335,7
3,0,0,2015-10-21 13:58:57,GILBERTO NASCIMENTO,CD01O318E,234,0,0,0,2015,000000,0,0
4,4,0,2017-12-06 01:31:27,RODRIGO MAIA,CD03O378E,MPV Nº 795/2017 - DTQ 11 - PV - ART. 7º DO TE...,222,19,1,2015,008100,369,123


In [5]:
def parse_header(header, include_vote_number=False):
    attrs = pd.Series({
        'body': header[:2],
        'legislative_session_number': header[2:4],
        'legislative_schedule': header[4:5],
        'session_number': header[5:8],
        'schedule': header[8:9],
    })
    return attrs

def rename_header_categorical_variables(dataframe):
    dtypes = {
        'legislative_schedule': 'category',
        'schedule': 'category',
        'body': 'category',
    }
    for col, col_type in dtypes.items():
        dataframe[col] = dataframe[col].astype(col_type)

    categories = {
        'O': 'ordinary_session',
        'E': 'special_session',
    }

    for col in ['schedule', 'legislative_schedule']:
        dataframe[col].cat.rename_categories(categories, inplace=True)

    dataframe['body'].cat.rename_categories({
        'CD': 'chamber_of_deputies',
        'CC': 'national_congress_chamber_of_deputies',
        'SF': 'national_congress_federal_senate',
    }, inplace=True)
    
    return dataframe

def dataframe_with_head_variables(dataframe):
    new_cols = dataframe['header'].apply(parse_header)
    dataframe = pd.concat(
        [dataframe.drop('header', axis=1), new_cols], axis=1)
    dataframe = rename_header_categorical_variables(dataframe)
    
    return dataframe

In [6]:
headers = dataframe_with_head_variables(headers)

Como a variável `headers` contém dois tipos de cabeçalhos distintos - de sessões e de votações - precisamos separar para tratar cada um das informações da forma que merece.

In [7]:
import re

def is_presence_row(row):
    return not not re.match(r'\d+', row['name'])

presence_rows = headers.apply(is_presence_row, axis=1)
presence_headers = headers[presence_rows]
vote_headers = headers[~presence_rows]
del(headers)

In [8]:
vote_headers.shape[0], presence_headers.shape[0]

(3618, 3983)

In [9]:
vote_specific_cols = [
    'abstention_votes',
    'blank_votes',
    'no_votes',
    'obstruction_votes',
    'president_votes',
    'vote_number',
    'votes',
    'yes_votes',
]
presence_headers = presence_headers \
    .drop(vote_specific_cols, axis=1) \
    .rename(columns={'name': 'congresspeople_present'})
presence_headers['congresspeople_present'] = \
    presence_headers['congresspeople_present'].astype(np.int)

In [10]:
vote_headers.head()

,abstention_votes,blank_votes,ending_time,first_president,name,no_votes,obstruction_votes,president_votes,term,vote_number,votes,yes_votes,body,legislative_schedule,legislative_session_number,schedule,session_number
2,3,0,2016-10-05 16:09:09,BETO MANSUR,PL Nº 4567/2016 - REQUERIMENTO DE ADIAMENTO ...,262,62,1,2015,007188,335,7,chamber_of_deputies,ordinary_session,02,special_session,241
4,4,0,2017-12-06 01:31:27,RODRIGO MAIA,MPV Nº 795/2017 - DTQ 11 - PV - ART. 7º DO TE...,222,19,1,2015,008100,369,123,chamber_of_deputies,ordinary_session,03,special_session,378
11,2,0,2015-06-10 18:29:51,EDUARDO CUNHA,PEC Nº 182/2007 - DTQ 27 - PMDB - PREFERÊNCIA...,268,0,1,2015,006368,385,114,chamber_of_deputies,ordinary_session,01,special_session,147
12,1,0,2017-05-16 15:25:20,JHC,MPV Nº 756/2016 - REQUERIMENTO DE RETIRADA DE...,255,61,1,2015,007538,336,18,chamber_of_deputies,ordinary_session,03,special_session,118
15,1,0,2016-12-14 22:51:22,BETO MANSUR,MPV Nº 744/2016 - DTQ 1: PT - EMENDA Nº 11,193,0,1,2015,007353,280,85,chamber_of_deputies,ordinary_session,02,special_session,334


In [11]:
presence_headers.head()

,ending_time,first_president,congresspeople_present,term,body,legislative_schedule,legislative_session_number,schedule,session_number
0,2017-09-04 20:27:45,CARLOS MANATO,345,2015,chamber_of_deputies,ordinary_session,03,special_session,242
1,2015-06-23 20:43:45,DANIEL VILELA,319,2015,chamber_of_deputies,ordinary_session,01,special_session,165
3,2015-10-21 13:58:57,GILBERTO NASCIMENTO,234,2015,chamber_of_deputies,ordinary_session,01,special_session,318
5,2015-09-22 02:19:44,RENAN CALHEIROS,72,2015,national_congress_federal_senate,ordinary_session,01,special_session,021
6,2017-09-26 20:34:57,DELEGADO EDSON MOREIRA,452,2015,chamber_of_deputies,ordinary_session,03,special_session,275


**Sanity checks**

É esperado que o número de votos de cada votação seja igual ao somatório de todos os tipos de votos possíveis.

In [12]:
vote_cols = [col for col in vote_headers.columns if col[-6:] == '_votes']
(vote_headers['votes'] == vote_headers[vote_cols].sum(axis=1)).value_counts()

True     3606
False      12
dtype: int64

In [13]:
vote_headers.to_csv('../data/chamber_of_deputies_votes.csv', index=False)
presence_headers.to_csv('../data/chamber_of_deputies_presences.csv', index=False)

---

## Detalhes de sessões

**Conteúdo arquivo**

```
aabbcddde 000000 xxxyyy zzzwwwfff ggg
```

| Linha                | Descrição                                    |
|----------------------|----------------------------------------------|
| **aabbcddde 000000** | conforme nomenclatura do cabeçalho           |
| **xxx**              | nome do parlamentar (40 posições)            |
| **yyy**              | Presente ou <------> (ausência) – 8 posições |
| **zzz**              | sigla do partido (10 posições)               |
| **www**              | nome da UF (25 posições)                     |
| **fff**              | código do parlamentar (3 posições)           |

In [14]:
paths = glob.glob('../data/sources/sessions/**/LP*')
presences = pd.DataFrame()
for index, path in enumerate(paths):
    if (index + 1) % 500 == 0:
        print('Reading {} out of {}'.format(index + 1, len(paths)))
    subset = pd.read_fwf(
        path, widths=[16, 40, 9, 10, 25, 3], header=None, encoding='iso-8859-1')
    subset['term'] = path.split('/')[-2]
    presences = presences.append(subset)

Reading 500 out of 3962
Reading 1000 out of 3962
Reading 1500 out of 3962
Reading 2000 out of 3962
Reading 2500 out of 3962
Reading 3000 out of 3962
Reading 3500 out of 3962


Definir os tipos corretos para cada um dos campos.

In [15]:
dtypes = {
    'header': np.str,
    'name': 'category',
    'status': 'category',
    'party': 'category',
    'state': 'category',
    'congressperson_id': 'category',
}
presences.rename(columns={
    0: 'header',
    1: 'name',
    2: 'status',
    3: 'party',
    4: 'state',
    5: 'congressperson_id',
}, inplace=True)
for col, col_type in dtypes.items():
    presences[col] = presences[col].astype(col_type)

In [16]:
presences['congressperson_id'] = \
    presences['congressperson_id'].apply(lambda val: str(int(val)) if val else None)

Renomear variáveis categóricas.

In [17]:
presences['status'].cat.rename_categories({
    'Presente': 'present',
    '<------>': 'absent',
}, inplace=True)

In [18]:
presences = dataframe_with_head_variables(presences)

In [19]:
presences.head()

,name,status,party,state,congressperson_id,term,body,legislative_schedule,legislative_session_number,schedule,session_number
0,ABEL MESQUITA JR.,present,PDT,Roraima,1,2015,chamber_of_deputies,ordinary_session,01,special_session,257
1,CARLOS ANDRADE,present,PHS,Roraima,3,2015,chamber_of_deputies,ordinary_session,01,special_session,257
2,EDIO LOPES,present,PMDB,Roraima,2,2015,chamber_of_deputies,ordinary_session,01,special_session,257
3,HIRAN GONÇALVES,present,PMN,Roraima,4,2015,chamber_of_deputies,ordinary_session,01,special_session,257
4,JHONATAN DE JESUS,present,PRB,Roraima,5,2015,chamber_of_deputies,ordinary_session,01,special_session,257


In [20]:
presences.to_csv('../data/chamber_of_deputies_presences_congresspeople.csv', index=False)

---

## Detalhes de votações

**Conteúdo arquivo**

```
aabbcddde ffffff xxx yyy  www ggg hhh iii
```

| Linha votação        | Descrição votação |
|----------------------|-------------------|
| **aabbcddde ffffff** | conforme nomenclatura do header |
| **xxx**              | nome do parlamentar (40 posições) |
| **yyy**              | qualidade de voto do parlamentar (Sim, Não, Abstenção, Obstrução, Branco) – 10 posições<br>Se a votação for secreta = Presente<br>Se não participou da votação = <-------> |
| **www**              | sigla do partido (10 posições) |
| **ggg**              | nome da uf (25 posições) |
| **hhh**              | código do parlamentar (3 posições) |

In [21]:
paths = glob.glob('../data/sources/sessions/**/LV*')
votes = pd.DataFrame()
for index, path in enumerate(paths):
    if (index + 1) % 500 == 0:
        print('Reading {} out of {}'.format(index + 1, len(paths)))
    subset = pd.read_fwf(
        path, widths=[16, 40, 10, 10, 25, 3], header=None, encoding='iso-8859-1')
    subset['term'] = path.split('/')[-2]
    subset['vote_number'] = path[-10:-4]
    votes = votes.append(subset)

Reading 500 out of 3616
Reading 1000 out of 3616
Reading 1500 out of 3616
Reading 2000 out of 3616
Reading 2500 out of 3616
Reading 3000 out of 3616
Reading 3500 out of 3616


In [22]:
dtypes = {
    'header': np.str,
    'name': 'category',
    'vote': 'category',
    'party': 'category',
    'state': 'category',
    'congressperson_id': 'category',
}
votes.rename(columns={
    0: 'header',
    1: 'name',
    2: 'vote',
    3: 'party',
    4: 'state',
    5: 'congressperson_id',
}, inplace=True)
for col, col_type in dtypes.items():
    votes[col] = votes[col].astype(col_type)

votes['congressperson_id'] = \
    votes['congressperson_id'].apply(lambda val: str(int(val)) if val else None)

In [23]:
votes['vote'].cat.rename_categories({
    'Sim': 'yes',
    'Não': 'no',
    'Abstenção': 'abstention',
    'Obstrução': 'obstruction',
    'Branco': 'blank',
    '<------->': 'absent',
}, inplace=True)

In [24]:
votes = dataframe_with_head_variables(votes)

In [25]:
votes.head()

,name,vote,party,state,congressperson_id,term,vote_number,body,legislative_schedule,legislative_session_number,schedule,session_number
0,CARLOS ANDRADE,no,PHS,Roraima,3,2015,006217,chamber_of_deputies,ordinary_session,01,ordinary_session,070
1,EDIO LOPES,no,PMDB,Roraima,2,2015,006217,chamber_of_deputies,ordinary_session,01,ordinary_session,070
2,HIRAN GONÇALVES,no,PMN,Roraima,4,2015,006217,chamber_of_deputies,ordinary_session,01,ordinary_session,070
3,JHONATAN DE JESUS,no,PRB,Roraima,5,2015,006217,chamber_of_deputies,ordinary_session,01,ordinary_session,070
4,SHÉRIDAN,no,PSDB,Roraima,8,2015,006217,chamber_of_deputies,ordinary_session,01,ordinary_session,070


In [26]:
votes.to_csv('../data/chamber_of_deputies_votes_congresspeople.csv', index=False)

---